## Données

In [2]:
pip install sdv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.2/145.2 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.9/51.9 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.9/169.9 kB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 46.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 8.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 14.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 87.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 81.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 4.1 MB/s eta 0:00

In [3]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import os
import time
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
from sdv.evaluation.single_table import run_diagnostic
from sdv.metadata import SingleTableMetadata
from sdv.single_table import CTGANSynthesizer
from sdv.metadata import SingleTableMetadata
from sdv.evaluation.single_table import get_column_plot
from sdv.evaluation.single_table import get_column_pair_plot
from sdv.evaluation.single_table import evaluate_quality
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

os.chdir('/home/onyxia/work/synthetic-data-sdc/')
os.environ["MLFLOW_TRACKING_URI"] = "https://projet-donnees-synthetiques-mlflow.user.lab.sspcloud.fr/" 

In [5]:
variables = ['sex','age','agegr','placesize','edu','socprof','marital','ls','depress','trust','trustfam','trustneigh','sport','nofriend','smoke','alcabuse','alcsol','wkabint','englang','height','weight','bmi']
original = pd.read_csv('SDV/df_original.csv', names = variables)
sample = pd.read_csv('SDV/df1_sample.csv', names = variables)
cart = pd.read_csv('SDV/df1_cart.csv', names = variables)
ctree = pd.read_csv('SDV/df1_ctree.csv', names = variables)
parametric = pd.read_csv('SDV/df1_parametric.csv', names = variables)
rf = pd.read_csv('SDV/df1_rf.csv', names = variables)
bag = pd.read_csv('SDV/df1_bag.csv', names = variables)

In [6]:
original.head()

,sex,age,agegr,placesize,edu,socprof,marital,ls,depress,trust,...,sport,nofriend,smoke,alcabuse,alcsol,wkabint,englang,height,weight,bmi
0,FEMALE,57,45-59,"URBAN 100,000-200,000",VOCATIONAL/GRAMMAR,RETIRED,MARRIED,PLEASED,6,ONE CAN`T BE TOO CAREFUL,...,NO,6,NO,NO,NO,NO,NONE,170,89,30.795848
1,MALE,20,16-24,RURAL AREAS,VOCATIONAL/GRAMMAR,PUPIL OR STUDENT,SINGLE,MOSTLY SATISFIED,0,IT`S DIFFICULT TO TELL,...,NO,4,NO,NO,NO,NO,ACTIVE,187,82,23.449341
2,FEMALE,18,16-24,"URBAN 500,000 AND OVER",VOCATIONAL/GRAMMAR,PUPIL OR STUDENT,SINGLE,PLEASED,0,MOST PEOPLE CAN BE TRUSTED,...,NO,20,NO,NO,NO,NO,ACTIVE,165,50,18.365473
3,FEMALE,78,65+,RURAL AREAS,PRIMARY/NO EDUCATION,RETIRED,WIDOWED,MIXED,16,ONE CAN`T BE TOO CAREFUL,...,YES,0,NO,NO,NO,NO,NONE,160,78,30.468750
4,FEMALE,54,45-59,"URBAN 100,000-200,000",VOCATIONAL/GRAMMAR,SELF-EMPLOYED,MARRIED,MOSTLY SATISFIED,4,ONE CAN`T BE TOO CAREFUL,...,YES,6,YES,NO,NO,NO,NONE,158,50,20.028842


In [7]:
num = ['age', 'depress', 'nofriend', 'height', 'weight', 'bmi']
fac = ['sex', 'agegr', 'placesize', 'edu', 'socprof', 'marital',
       'ls', 'trust', 'trustfam', 'trustneigh', 'sport', 'smoke',
       'alcabuse', 'alcsol', 'wkabint', 'englang']

original[num].describe()

,age,depress,nofriend,height,weight,bmi
count,4603.000000,4603.000000,4603.000000,4603.000000,4603.000000,4603.000000
mean,47.449055,4.509016,6.916142,168.732566,73.999566,25.951256
std,18.465555,4.230392,7.214682,9.276404,15.034077,4.731237
min,16.000000,0.000000,0.000000,116.000000,37.000000,12.962963
25%,32.000000,1.000000,3.000000,162.000000,62.000000,22.598140
50%,48.000000,4.000000,5.000000,168.000000,73.000000,25.535446
75%,61.000000,7.000000,10.000000,175.000000,84.000000,28.727377
max,97.000000,21.000000,99.000000,202.000000,150.000000,76.127550


In [8]:
original[fac].describe()

,sex,agegr,placesize,edu,socprof,marital,ls,trust,trustfam,trustneigh,sport,smoke,alcabuse,alcsol,wkabint,englang
count,4603,4603,4603,4603,4603,4603,4603,4603,4603,4603,4603,4603,4603,4603,4603,4603
unique,2,6,6,4,9,6,7,3,3,3,2,2,2,2,3,3
top,FEMALE,45-59,RURAL AREAS,VOCATIONAL/GRAMMAR,RETIRED,MARRIED,PLEASED,ONE CAN`T BE TOO CAREFUL,YES,YES,YES,NO,NO,NO,NO,NONE
freq,2596,1243,2221,1480,1132,2752,1799,3515,4127,2731,2981,3415,4304,4448,4297,3174


## Génération des données

In [23]:
original['t'] = 0
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(original)
metadata

{
    "columns": {
        "sex": {
            "sdtype": "categorical"
        },
        "age": {
            "sdtype": "numerical"
        },
        "agegr": {
            "sdtype": "categorical"
        },
        "placesize": {
            "sdtype": "categorical"
        },
        "edu": {
            "sdtype": "categorical"
        },
        "socprof": {
            "sdtype": "categorical"
        },
        "marital": {
            "sdtype": "categorical"
        },
        "ls": {
            "sdtype": "categorical"
        },
        "depress": {
            "sdtype": "numerical"
        },
        "trust": {
            "sdtype": "categorical"
        },
        "trustfam": {
            "sdtype": "categorical"
        },
        "trustneigh": {
            "sdtype": "categorical"
        },
        "sport": {
            "sdtype": "categorical"
        },
        "nofriend": {
            "sdtype": "numerical"
        },
        "smoke": {
            "sdtype": "categoric

## Grid search

In [169]:
import itertools
import yaml
from yaml.loader import SafeLoader
import mlflow
from sdv.single_table import CTGANSynthesizer
import mlflow.pyfunc
import joblib
import os
import time
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import pandas as pd

# Charger les configurations du fichier YAML
with open("SDV/config.yml") as f:
    config = yaml.load(f, Loader=SafeLoader)

class CTGANWrapper(mlflow.pyfunc.PythonModel):
    def __init__(self):
        self.model = None

    def load_context(self, context):
        self.model = joblib.load(context.artifacts["model_path"])

    def predict(self, context, model_input):
        return self.model.sample(len(model_input))

remote_server_uri = os.environ["MLFLOW_TRACKING_URI"]
experiment_name = 'Default'
run_name = 'grid_search'

mlflow.set_tracking_uri(remote_server_uri)
mlflow.set_experiment(experiment_name)

batch_size_values = config['batch_size']
discriminator_lr_values = config['discriminator_lr']
generator_lr_values = config['generator_lr']

param_combinations = list(itertools.product(batch_size_values, discriminator_lr_values, generator_lr_values))

for param_set in param_combinations:
    batch_size, discriminator_lr, generator_lr = param_set
    run_name = f"epochs_500_bs_{batch_size}_disc_lr_{discriminator_lr}_gen_lr_{generator_lr}"

    with mlflow.start_run(run_name=run_name):
        # Log des paramètres de configuration
        mlflow.log_param('batch_size', batch_size)
        mlflow.log_param('discriminator_lr', discriminator_lr)
        mlflow.log_param('generator_lr', generator_lr)
        mlflow.log_param('run_name', run_name)

        # Entraînement
        start_time1 = time.time()
        ctgan = CTGANSynthesizer(metadata, verbose=True, batch_size=batch_size, discriminator_lr=discriminator_lr, generator_lr=generator_lr, epochs=500)
        ctgan.fit(original)
        end_time1 = time.time()

        elapsed_time1 = end_time1 - start_time1
        mlflow.log_metric("elapsed_time", elapsed_time1)
        print(f"Temps pris pour entraîner le modèle avec batch_size={batch_size}, discriminator_lr={discriminator_lr}, generator_lr={generator_lr}: {elapsed_time1} secondes")

        # Calcul du pMSE
        synthetique = ctgan.predict(original)
        synthetique['t'] = 1
        original['t'] = 0
        c = synthetique.shape[0] / (synthetique.shape[0] + original.shape[0])
        
        df_comb = pd.concat([original, synthetique], ignore_index=True)
        df_comb_enc = pd.get_dummies(df_comb, columns=fac)
        X = df_comb_enc.drop(columns=['t'])
        y = df_comb_enc['t']
        
        model = DecisionTreeClassifier(random_state=1)
        model.fit(X, y)
        prop = model.predict(X)
        pMSE = mean_squared_error(c, prop)
        print(f'pMSE: {pMSE}')

        # Log des pertes du générateur et du discriminateur
        loss = ctgan.get_loss_values()
        generator_losses = loss['Generator Loss'].tolist()
        discriminator_losses = loss['Discriminator Loss'].tolist()
        for epoch, loss in enumerate(generator_losses):
            mlflow.log_metric("generator_loss", loss, step=epoch)
        for epoch, loss in enumerate(discriminator_losses):
            mlflow.log_metric("discriminator_loss", loss, step=epoch)

        # Sauvegarde du modèle
        model_path = f"ctgan_model_{run_name}.pkl"
        joblib.dump(ctgan, model_path)
        mlflow.pyfunc.log_model(
            artifact_path="ctgan_model",
            python_model=CTGANWrapper(),
            artifacts={"model_path": model_path},
        )
    mlflow.end_run()

Gen. (-8.25) | Discrim. (6.76): 100%|██████████| 500/500 [06:42<00:00,  1.24it/s]       


Temps pris pour entraîner le modèle avec batch_size=1000, discriminator_lr=0.1, generator_lr=0.1: 408.0361828804016 secondes
pMSE: 0.012926352378883336


Gen. (-104.38) | Discrim. (2.09): 100%|██████████| 500/500 [06:26<00:00,  1.29it/s]    


Temps pris pour entraîner le modèle avec batch_size=1000, discriminator_lr=0.1, generator_lr=0.0001: 391.853634595871 secondes
pMSE: 0.01140560504019118


Gen. (-1540.36) | Discrim. (-453.44): 100%|██████████| 500/500 [06:20<00:00,  1.31it/s]


...

Temps pris pour entraîner le modèle avec batch_size=100, discriminator_lr=1e-08, generator_lr=0.1: 2699.2863824367523 secondes
pMSE: 0.009884857701499022


Gen. (-0.06) | Discrim. (0.07): 100%|██████████| 500/500 [37:13<00:00,  4.47s/it]  


Temps pris pour entraîner le modèle avec batch_size=100, discriminator_lr=1e-08, generator_lr=0.0001: 2239.33203792572 secondes
pMSE: 0.011188355420378014


Gen. (1.22) | Discrim. (-0.04): 100%|██████████| 500/500 [35:20<00:00,  4.24s/it] 


Temps pris pour entraîner le modèle avec batch_size=100, discriminator_lr=1e-08, generator_lr=1e-08: 2125.996749162674 secondes
pMSE: 0.013360851618509667


## Tests pMSE

In [126]:
ctgan_test = mlflow.pyfunc.load_model("runs:/d682b1f373e14cedbcac2ddff64ab9d2/ctgan_model")

In [170]:
original['t'] = 0
synthetique = ctgan_test.predict(original)
synthetique['t'] = 1
c = synthetique.shape[0] / (synthetique.shape[0] + original.shape[0])

df_comb = pd.concat([original, synthetique], ignore_index=True)
df_comb_enc = pd.get_dummies(df_comb, columns=fac)
X = df_comb_enc.drop(columns=['t'])
y = df_comb_enc['t']

model = DecisionTreeClassifier(random_state=1)
model.fit(X, y)
prop = model.predict(X)
pMSE = mean_squared_error(c, prop)
print(f'pMSE: {pMSE}')

pMSE: 0.011948729089724093


## Comparatif avec CART

In [167]:
original['t'] = 0
cart['t'] = 1
c1 = cart.shape[0] / (cart.shape[0] + original.shape[0])

df_comb1 = pd.concat([original, cart], ignore_index=True)
df_comb_enc1 = pd.get_dummies(df_comb1, columns=fac)
X1 = df_comb_enc1.drop(columns=['t'])
y1 = df_comb_enc1['t']

model1 = DecisionTreeClassifier(random_state=1)
model1.fit(X1, y1)
prop1 = model1.predict_proba(X1)
pMSE1 = mean_squared_error(c1, prop1)
print(f'pMSE: {pMSE1}')

pMSE: 0.1596784705626765


## Combinaison unique

In [ ]:
import itertools
import yaml
from yaml.loader import SafeLoader
import mlflow
from sdv.single_table import CTGANSynthesizer
import mlflow.pyfunc
import joblib
import os
import time
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import pandas as pd

# Charger les configurations du fichier YAML
with open("SDV/config.yml") as f:
    config = yaml.load(f, Loader=SafeLoader)

class CTGANWrapper(mlflow.pyfunc.PythonModel):
    def __init__(self):
        self.model = None

    def load_context(self, context):
        self.model = joblib.load(context.artifacts["model_path"])

    def predict(self, context, model_input):
        return self.model.sample(len(model_input))

remote_server_uri = os.environ["MLFLOW_TRACKING_URI"]
experiment_name = 'Default'
run_name = 'run'

mlflow.set_tracking_uri(remote_server_uri)
mlflow.set_experiment(experiment_name)

batch_size = 10
discriminator_lr = 0.0001
generator_lr = 0.0001

run_name = f"epochs_500_bs_{batch_size}_disc_lr_{discriminator_lr}_gen_lr_{generator_lr}"

with mlflow.start_run(run_name=run_name):
    # Log des paramètres de configuration
    mlflow.log_param('batch_size', batch_size)
    mlflow.log_param('discriminator_lr', discriminator_lr)
    mlflow.log_param('generator_lr', generator_lr)
    mlflow.log_param('run_name', run_name)

    # Entraînement
    start_time1 = time.time()
    ctgan = CTGANSynthesizer(metadata,
                             verbose=True,
                             batch_size=batch_size, 
                             discriminator_lr=discriminator_lr,
                             generator_lr=generator_lr,
                             epochs=500)
    ctgan.fit(original)
    end_time1 = time.time()

    elapsed_time1 = end_time1 - start_time1
    mlflow.log_metric("elapsed_time", elapsed_time1)
    print(f"Temps pris pour entraîner le modèle avec batch_size={batch_size}, discriminator_lr={discriminator_lr}, generator_lr={generator_lr}: {elapsed_time1} secondes")

    # Log des pertes du générateur et du discriminateur
    loss = ctgan.get_loss_values()
    generator_losses = loss['Generator Loss'].tolist()
    discriminator_losses = loss['Discriminator Loss'].tolist()
    for epoch, loss in enumerate(generator_losses):
        mlflow.log_metric("generator_loss", loss, step=epoch)
    for epoch, loss in enumerate(discriminator_losses):
        mlflow.log_metric("discriminator_loss", loss, step=epoch)

    # Sauvegarde du modèle
    model_path = f"ctgan_model_{run_name}.pkl"
    joblib.dump(ctgan, model_path)
    mlflow.pyfunc.log_model(
        artifact_path="ctgan_model",
        python_model=CTGANWrapper(),
        artifacts={"model_path": model_path},
    )
    mlflow.end_run()

Gen. (0.54) | Discrim. (0.77):   0%|          | 1/500 [00:48<6:39:15, 48.01s/it]